In [61]:
# Imports
from matplotlib import pyplot as plt
import librosa
import torch
import torch.nn as nn
from torchsummary import summary
import torchaudio
import soundfile as sf
from scipy import signal
import getpass
import pandas as pd
import numpy as np
import sys
import os
if getpass.getuser()=="joanna.luberadzka":
    projectdir="/Users/joanna.luberadzka/Documents/VAE-IR/"
    datadir="/Users/joanna.luberadzka/Documents/Data/IR_Arni_upload_numClosed_0-5/"
elif getpass.getuser()=="ubuntu":
    projectdir="/home/ubuntu/joanna/VAE-IR/"
    datadir="/home/ubuntu/Data/IR_Arni_upload_numClosed_0-5/"

sys.path.insert(0, projectdir+'src/')
from scipy.io import wavfile
from IPython.display import Audio
# my modules:
import helpers
import importlib
importlib.reload(helpers)

<module 'helpers' from '/Users/joanna.luberadzka/Documents/VAE-IR/src/helpers.py'>

In [2]:
mydata = pd.read_csv(datadir+"combinations_setup.csv",delimiter=';', index_col=0)
# find combination numbers corresponding to 0-5 reflective panels
# from the total of 55 panels
n_closed=55-mydata[mydata.sum(axis=1)>49].sum(axis=1).to_numpy()
avail_comb=mydata[mydata.sum(axis=1)>49].index.to_numpy()

In [58]:
# get one random impulse response from the database
# note: NUMCOMB depends on NUMCLOSED
rnd_idx=np.random.randint(len(avail_comb))
NUMCOMB=avail_comb[rnd_idx]
NUMCLOSED=int(n_closed[rnd_idx])
NUMMIC=np.random.randint(1,5)
NUMSWEEP=np.random.randint(1,5)
filename=f"IR_numClosed_{NUMCLOSED}_numComb_{NUMCOMB}_mic_{NUMMIC}_sweep_{NUMSWEEP}.wav"

In [65]:
newlist = pd.DataFrame(columns=['filename', 'T30', 'DRR'])
for filename in os.listdir(datadir):
    f = os.path.join(datadir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        x, fs=librosa.load(datadir+filename)

KeyboardInterrupt: 

In [59]:
# load chosen RIR
h, sr_h= torchaudio.load(datadir+filename)
# load anechoic sound file
x, sr_x=torchaudio.load(projectdir+"anechoic.wav")
# resample anechoic signal
x=torchaudio.transforms.Resample(sr_x,sr_h)(x)
# convolve anechoic signal with RIR: y=x*h
y = signal.fftconvolve(x, h, mode = 'full')
# set signal levels
x=x.numpy()
x=helpers.set_level(x,-30)
y=helpers.set_level(y,-30)
# save signals
sf.write("convolved.wav", y.T, sr_h, subtype='PCM_24')
sf.write("anechoic.wav", x.T, sr_h, subtype='PCM_24')

In [40]:
Audio('convolved.wav')

In [41]:
Audio('anechoic.wav')

In [57]:
# Plot anechoic signal, convolved signal and impulse response

# plt.figure(1)
# plt.plot(x.T,label='anechoic')
# plt.plot(y.T,label='convolved')
# plt.legend()
# plt.show()

# plt.figure(1)
# plt.plot(h.T,label='RIR')
# plt.legend()
# plt.show()